# Паттерн абстрактная фабрика

В данном файле будет продемонстрирован ппример реализации паттерна абстрактная фабрика.

### Задача

Создайте фабрики по производству файлов отчётов в форматах HTML и Markdown.
<hr/>
Для начала импортируем библиотеку `abc` для работы с абстрактными классами.

In [15]:
import abc  # для работы с абстрактным классом
import sys  # для возможности вывести результаты работы прямо в notebook

### Тестовая функция
Перед созданием фабрик напишем функцию, для тестирования фабрик: `test`

In [16]:
def test():

    mdFilename = "report.md"
    HTMLFilename = "report.html"

    txtreport = create_report(MDreportFactory())
    txtreport.save(mdFilename)
    print("Сохранено:", mdFilename)

    HTMLreport = create_report(HTMLreportFactory())
    HTMLreport.save(HTMLFilename)
    print("Сохранено:", HTMLFilename)

В функции `test` определяються имена для файлов с результатами работы - `report.md` для отчёта в формате Markdown и `report.html` для отчета в HTML формате.

После этого происходи создание отчёта при помощи функции `create_report`, которой передаётся непосредственно фабрика. В первом случае — `MDreportFactory` - фабрика производства Markdown отчётов. Во втором — `HTMLreportFactory` для HTML.

Создав отчёт сохраняем его в соотвсетствующий файл.

### Создание отчёта

Перейдём к функции `create_report` которая и будет создавать отчёт. Параметром данной функции являеться фабрика, которая будет «производить» отчёт и его составные компоненты  

In [17]:
def create_report(factory):
    # создаём отчёт
    report = factory.make_report("Report")           
    # создаём первую часть отчёта
    chapter1 = factory.make_chapter("Chapter one")
    # создаём вторую часть отчёта
    chapter2 = factory.make_chapter("Chapter two")

  # первая часть отчёта
    # добавляем текст
    chapter1.add("chapter 1 text")
    # создаём ссылку
    link = factory.make_link(
        "coursera", "https://ru.coursera.org")  
    # добавляем ссылку
    chapter1.add(link)

  # вторая часть отчёта
    # добавляем текст
    chapter2.add("Chapter 2 header\n\n")
    # создаём картинку
    img = factory.make_img("image",
                           "https://blog.coursera.org/wp-content/uploads/2017/07/coursera-fb.png")
    # создаём ссылку из картинки
    link = factory.make_link(img, "https://ru.coursera.org")
    # добавляем ссылку
    chapter2.add(link)
    # добавляем текст
    chapter2.add("\n\nChapter 2 footer")

  # отчёт
    report.add(chapter1)  # добавляем первую часть
    report.add(chapter2)  # добавляем вторую часть
    return report         # возвращаем отчёт

### Абстрактная фабрика

Создадим абстрактную фабрику с классовыми методами. Каждый метод создаёт экземпляр соответствующего подкласса: `Report` для отчёта, с названием `Title`; `Chapter` для главы c заголовком `Caption`; `Link` для преобразования объекта `obj` (может быть просто текст) в ссылку на `href`; `Img` для создания изображения с описанием `alt_text`, находящегося по адресу `src`.  

In [18]:
class ReportFactory:

    @classmethod                       # Отчёт
    def make_report(Class, title):
        return Class.Report(title)

    @classmethod                       # Часть
    def make_chapter(Class, caption):
        return Class.Chapter(caption)

    @classmethod                       # Ссылка
    def make_link(Class, obj, href):
        return Class.Link(obj, href)

    @classmethod                       # Картинка
    def make_img(Class, alt_text, src):
        return Class.Img(alt_text, src)

### Markdown фабрика

Перейдём к реализации подклассов `Report`, `Chapter`, `Link` и `Img`. Для фабрики, производящей Markdown отчёты.

In [19]:
class MDreportFactory(ReportFactory):

    # Отчёт
    class Report:

        # Создание отчёта
        def __init__(self, title):
            self.parts = []  # Создаём список всех частей отчёта
            # и сразу добавляем в список название
            self.parts.append(f"# {title}\n\n")

    # Добавление части
        def add(self, part):
            self.parts.append(part)  # Просто добавляем в список

    # Функция сохранения отчёта
        def save(self, filenameOrFile):
            # Определяем, что было передано:
            # имя файла (file = None)
            # или файловая переменная (file = filenameOrFile).
            file = None if isinstance(filenameOrFile, str) else filenameOrFile

        # Пытаемся произвести сохранения отчёта
            try:
                if file is None:
                    # открываем файл для записи
                    file = open(filenameOrFile, "w", encoding="utf-8")

            # Печатаем в file все части, преобразованные к строке
                print('\n'.join(map(str, self.parts)), file=file)

        # Закрываем файл, если мы его открывали
            finally:
                if isinstance(filenameOrFile, str) and file is not None:
                    file.close()

# Часть
    class Chapter:

        # Создание части
        def __init__(self, caption):
            self.caption = caption  # Сохраняем заглавие
            self.objects = []       # Создаём список содержимого данной части отчёта

    # Добавление содержимого в отчёт
        def add(self, obj):
            self.objects.append(obj)  # Просто добавляем в список

    # Преобразование к строке: отделяем заглавие, после чего
    # присоединяем всё содержимое, преобразованное к строке.
        def __str__(self):
            return f'## {self.caption}\n\n' + ''.join(map(str, self.objects))

# Ссылка
    class Link:
        # Создание ссылки - сохранение объекта и адреса
        def __init__(self, obj, href):
            self.obj = obj
            self.href = href

    # Преобразование к строке - вывод ссылки в Markdown виде:
        def __str__(self):
            return f'[{self.obj}]({self.href})'

# Изображение
    class Img:
        # Создание изображения - сохранение его описания и расположения
        def __init__(self, alt_text, src):
            self.alt_text = alt_text
            self.src = src

    # Преобразование к строке - вывод ссылки в Markdown виде:
        def __str__(self):
            return f'![{self.alt_text}]({self.src})'

### HTML фабрика

Перейдём к реализации подклассов `Report`, `Chapter`, `Link` и `Img`. Для фабрики, производящей HTML отчёты.

In [20]:
class HTMLreportFactory(ReportFactory):

    # Отчёт
    class Report:

        # Создание отчёта
        def __init__(self, title):
            self.title = title
            # Создаём список всех частей отчёта и добавляем в него шапку:
            self.parts = []
            self.parts.append("<html>")  # Открывающий тег
            self.parts.append("<head>")  # Раздел заголовков
            self.parts.append("<title>" + title + "</title>")  # Название
            self.parts.append("<meta charset=\"utf-8\">")
            self.parts.append("</head>")
            self.parts.append("<body>")  # Начало содержимого

    # Добавление части
        def add(self, part):
            self.parts.append(part)  # Просто добавляем в список

    # Функция сохранения отчёта
        def save(self, filenameOrFile):
            self.parts.append("</body></html>")  # Добавляем закрывающие тэги

            # Определяем, что было передано: имя файла (file = None) или файловая переменная (file = filenameOrFile).
            file = None if isinstance(filenameOrFile, str) else filenameOrFile

        # Пытаемся произвести сохранение отчёта
            try:
                if file is None:
                    # открываем файл для записи
                    file = open(filenameOrFile, "w", encoding="utf-8")

            # Печатаем в file все части, преобразованные к строке
                print(''.join(map(str, self.parts)), file=file)

        # Закрываем файл, если мы его открывали
            finally:
                if isinstance(filenameOrFile, str) and file is not None:
                    file.close()

# Часть
    class Chapter:

        # Создание части
        def __init__(self, caption):
            self.caption = caption  # Сохраняем заглавие
            self.objects = []       # Создаём список содержимого данной части отчёта

    # Добавление содержимого в отчёт
        def add(self, obj):
            self.objects.append(obj)  # Просто добавляем в список

    # Преобразование к строке: обрамляем заглавие тэгами, после чего
    # присоединяем всё содержимое, преобразованное к строке.
        def __str__(self):
            ch = f'<h1>{self.caption}</h1>'
            return ch + ''.join(map(str, self.objects))

# Ссылка
    class Link:
        # Создание ссылки - сохранение объекта и адреса
        def __init__(self, obj, href):
            self.obj = obj
            self.href = href

    # Преобразование к строке - вывод ссылки в HTML виде:
        def __str__(self):
            return f'<a href = "{self.href}">{self.obj}</a>'

# Изображение
    class Img:
        # Создание изображения - сохранение его описания и расположения
        def __init__(self, alt_text, src):
            self.alt_text = alt_text
            self.src = src

    # Преобразование к строке - вывод ссылки в виде HTML виде:
        def __str__(self):
            return f'<img alt = "{self.alt_text}", src = "{self.src}"/>'

### Проверка работы

In [21]:
from IPython.display import display, Markdown, HTML

test()
display(Markdown('# <span style="color:red">report.md</span>'))
display(Markdown(filename="report.md"))
display(Markdown('# <span style="color:red">report.html</span>'))
display(HTML(filename="report.html"))

Сохранено: report.md
Сохранено: report.html


# <span style="color:red">report.md</span>

# Report


## Chapter one

chapter 1 text[coursera](https://ru.coursera.org)
## Chapter two

Chapter 2 header

[![image](https://blog.coursera.org/wp-content/uploads/2017/07/coursera-fb.png)](https://ru.coursera.org)

Chapter 2 footer


# <span style="color:red">report.html</span>